# EDA of data

## 0. Import of libreries


In [52]:
!pip install lightgbm

In [2]:
#--- LIBRERIAS
import pickle as pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
import sklearn as sklearn
from scipy import stats, special
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import json

In [3]:
# importing the NLTK related libraries and functions along with evaluation metircs
import nltk
import string
nltk.download("all")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\ariss\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\ariss\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\ariss\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\ariss\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\ariss\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping grammars\basque_grammars.zip.
[nltk_data]   

In [69]:
df = pd.read_csv("train_challenge.csv")

In [70]:
df.head()

,name,description,maker,trademark
0,Cerveza Otro Mundo 500 Ml,Cerveza Otro Mundo 500 Ml,NaN,Otro Mundo
1,Vino San Telmo Varietales X750Ml.,Vino San Telmo Varietales X750Ml.,NaN,San Telmo
2,Mayonesa Natura 1000cc,Mayonesa Natura con jugo de limon sachet x 100...,gallega,Natura
3,Brahma Chopp 354cc x 6,Brahma Chopp 354cc x 6,NaN,Brahma
4,Mayonesa HellmannS Light Doypack 237 Gr,Mayonesa HellmannS Light Doypack 237 Gr,NaN,Hellmann´S


In [71]:
df.shape

(10000, 4)

In [72]:
datatypes = df.dtypes 
  
datatypes

name           object
description    object
maker          object
trademark      object
dtype: object

In [73]:
df.describe()

,name,description,maker,trademark
count,10000,10000,3418,9965
unique,6180,6226,529,1201
top,Vino Emilia Nieto Senetiner Varietales X 750 Ml.,Vino Emilia Nieto Senetiner Varietales X 750 Ml.,gallega,Arcor
freq,11,11,271,198


# 1. Identify categories

In [74]:
df["name"]= df["name"].str.lower() 

sub ='cerv'
sub2= 'vin'
sub3='whisk'
sub4='champa'
sub5='espumante'
sub6='frizze'
start = 0
  
df["flg_cerveza"]= df["name"].str.find(sub, start)
df["flg_vino"]= df["name"].str.find(sub2, start)
df["flg_whisky"]= df["name"].str.find(sub3, start)
df["flg_champagne"]= df["name"].str.find(sub4, start)
df["flg_espumante"]= df["name"].str.find(sub5, start)
df["flg_frizze"]= df["name"].str.find(sub6, start)

df["flg_alcohol"]=np.where(df['flg_cerveza']!=-1 , 0,
                   np.where(df['flg_vino']!=-1, 0,
                   np.where(df['flg_whisky']!=-1, 0,
                   np.where(df['flg_champagne']!=-1,0,
                    np.where(df['flg_espumante']!=-1,0,
                     np.where(df['flg_frizze']!=-1,0,-1))))))



sub ='lec'
sub2= 'ques'
sub3='yogu'
sub4='mortadel'
sub5='lac'
sub6='fór'

df["flg_leche"]= df["name"].str.find(sub, start)
df["flg_queso"]= df["name"].str.find(sub2, start)
df["flg_yogur"]= df["name"].str.find(sub3, start)
df["flg_mortadela"]= df["name"].str.find(sub4, start)
df["flg_lacteo"]= df["name"].str.find(sub5, start)
df["flg_formula"]= df["name"].str.find(sub6, start)


df["flg_lacteo"]=np.where(df['flg_leche']!=-1 , 0,
                   np.where(df['flg_queso']!=-1, 0,
                   np.where(df['flg_yogur']!=-1, 0,
                    np.where(df['flg_mortadela']!=-1, 0,
                    np.where(df['flg_lacteo']!=-1, 0,
                    np.where(df['flg_formula']!=-1, 0,-1))))))

sub ='agua'
sub2= 'jugo'
sub3='limonada'
sub4='gatorade'
sub5='bebid'

df["flg_agua"]= df["name"].str.find(sub, start)
df["flg_jugo"]= df["name"].str.find(sub2, start)
df["flg_limonada"]= df["name"].str.find(sub3, start)
df["flg_gatorade"]= df["name"].str.find(sub4, start)
df["flg_bebida"]= df["name"].str.find(sub5, start)

df["flg_bebida"]=np.where(df['flg_agua']!=-1 , 0,
                   np.where(df['flg_jugo']!=-1,  0,
                    np.where(df['flg_limonada']!=-1,  0,
                  np.where(df['flg_gatorade']!=-1,  0,
                np.where(df['flg_bebida']!=-1,  0,-1)))))


sub ='fiambre'
sub2= 'pan'
sub3='choc'
sub4='salchi'
sub5='adere'
sub6='vinag'
sub7='aceit'
sub8='gom'
sub9='chicl'
sub10='galle'
sub11='turr'
sub12='caram'
sub13='sals'
sub14='alfaj'
sub15='mayon'
sub16='ketch'
sub22='mostaza'
sub17='dulc'
sub18='oblea'
sub19='chupet'
sub20='embutid'
sub21='aceto'
sub23='postre'
sub24='huevo'
sub25='pastilla'
sub26='jam'
sub27='bomb'
sub28='chimi'
sub29='confi'
sub30='table'
sub31='tostad'
sub32='reboza'
sub33='sala'
sub34='bocad'
sub35='bague'
sub36='bon o'
sub37='Pan'
sub38='bondio'
sub39='garra'
start = 0
  
df["flg_fiambre"]= df["name"].str.find(sub, start)
df["flg_pan"]= df["name"].str.find(sub2, start)
df["flg_choco"]= df["name"].str.find(sub3, start)
df["flg_salchicha"]= df["name"].str.find(sub4, start)
df["flg_aderezo"]= df["name"].str.find(sub5, start)
df["flg_vinagre"]= df["name"].str.find(sub6, start)
df["flg_aceite"]= df["name"].str.find(sub7, start)
df["flg_goma"]= df["name"].str.find(sub8, start)
df["flg_chicle"]= df["name"].str.find(sub9, start)
df["flg_galleta"]= df["name"].str.find(sub10, start)
df["flg_turron"]= df["name"].str.find(sub11, start)
df["flg_caramelos"]= df["name"].str.find(sub12, start)
df["flg_salsa"]= df["name"].str.find(sub13, start)
df["flg_alfajor"]= df["name"].str.find(sub14, start)
df["flg_mayonesa"]= df["name"].str.find(sub15, start)
df["flg_ketchup"]= df["name"].str.find(sub16, start)
df["flg_dulce"]= df["name"].str.find(sub17, start)
df["flg_oblea"]= df["name"].str.find(sub18, start)
df["flg_chupetin"]= df["name"].str.find(sub19, start)
df["flg_embutido"]= df["name"].str.find(sub20, start)
df["flg_aceto"]= df["name"].str.find(sub21, start)
df["flg_mostaza"]= df["name"].str.find(sub22, start)
df["flg_postre"]= df["name"].str.find(sub23, start)
df["flg_huevo"]= df["name"].str.find(sub24, start)
df["flg_pastilla"]= df["name"].str.find(sub25, start)
df["flg_jamon"]= df["name"].str.find(sub26, start)
df["flg_bombones"]= df["name"].str.find(sub27, start)
df["flg_chimichurri"]= df["name"].str.find(sub28, start)
df["flg_confite"]= df["name"].str.find(sub29, start)
df["flg_tableta"]= df["name"].str.find(sub30, start)
df["flg_tostada"]= df["name"].str.find(sub31, start)
df["flg_rebozador"]= df["name"].str.find(sub32, start)
df["flg_salame"]= df["name"].str.find(sub33, start)
df["flg_bocadito"]= df["name"].str.find(sub34, start)
df["flg_bague"]= df["name"].str.find(sub35, start)
df["flg_bono"]= df["name"].str.find(sub36, start)
df["flg_pan2"]= df["trademark"].str.find(sub37, start)
df["flg_bondiola"]= df["name"].str.find(sub38, start)
df["flg_garrapi"]= df["name"].str.find(sub39, start)


df["flg_almacen"]=np.where(df['flg_fiambre']!=-1 , 0,
                   np.where(df['flg_pan']!=-1, 0,
                   np.where(df['flg_choco']!=-1, 0,
                   np.where(df['flg_salchicha']!=-1,0,
                   np.where(df['flg_aderezo']!=-1,0,
                   np.where(df['flg_vinagre']!=-1,0,
                   np.where(df['flg_aceite']!=-1,0,
                   np.where(df['flg_goma']!=-1,0,
                   np.where(df['flg_chicle']!=-1,0,
                   np.where(df['flg_galleta']!=-1,0,
                   np.where(df['flg_turron']!=-1,0,
                   np.where(df['flg_caramelos']!=-1,0,
                   np.where(df['flg_salsa']!=-1,0,
                   np.where(df['flg_alfajor']!=-1,0,
                   np.where(df['flg_mayonesa']!=-1,0,
                   np.where(df['flg_ketchup']!=-1,0,
                   np.where(df['flg_dulce']!=-1,0,
                   np.where(df['flg_oblea']!=-1,0,
                   np.where(df['flg_chupetin']!=-1,0,
                   np.where(df['flg_embutido']!=-1,0,
                   np.where(df['flg_aceto']!=-1,0,
                   np.where(df['flg_mostaza']!=-1,0,
                   np.where(df['flg_postre']!=-1,0,
                   np.where(df['flg_huevo']!=-1,0,
                   np.where(df['flg_pastilla']!=-1,0,
                   np.where(df['flg_jamon']!=-1,0,
                   np.where(df['flg_bombones']!=-1,0,
                   np.where(df['flg_chimichurri']!=-1,0,
                   np.where(df['flg_confite']!=-1,0,
                   np.where(df['flg_tableta']!=-1,0,
                   np.where(df['flg_tostada']!=-1,0,
                   np.where(df['flg_rebozador']!=-1,0,
                   np.where(df['flg_salame']!=-1,0,
                    np.where(df['flg_bocadito']!=-1,0,
                     np.where(df['flg_bague']!=-1,0,
                     np.where(df['flg_bono']!=-1,0,
                          np.where(df['flg_pan2']!=-1,0,
                            np.where(df['flg_bondiola']!=-1,0,
                           np.where(df['flg_garrapi']!=-1,0,-1)))))))))))))))))))))))))))))))))))))))

df["flg_almacen"]=np.where(df['flg_almacen']==0,df['flg_almacen'],
                   np.where(df["name"].str.find('tarte', start)!=-1, 0,
                    np.where(df["name"].str.find('grisi', start)!=-1, 0,
                      np.where(df["name"].str.find('leber', start)!=-1, 0,-1))))

df["flg_alcohol"]=np.where(df['flg_alcohol']==0,df['flg_alcohol'],
                   np.where(df["name"].str.find('brah', start)!=-1, 0,
                   np.where(df["name"].str.find('budwi', start)!=-1,0,
                    np.where(df["name"].str.find('andes o', start)!=-1,0,-1))))

df["flg_bebida"]=np.where(df['flg_bebida']==0,df['flg_bebida'],
                   np.where(df["name"].str.find('soda', start)!=-1, 0,
                   np.where(df["name"].str.find('gaseo', start)!=-1,0,-1)))



df["sub_categoria"]=np.where(df['flg_leche']!=-1, 'leches',
                   np.where(df['flg_queso']!=-1, 'quesos',
                   np.where(df['flg_yogur']!=-1, 'yogures',
                   np.where(df['flg_cerveza']!=-1, 'cervezas',
                   np.where(df['flg_vino']!=-1, 'vino',
                   np.where(df['flg_whisky']!=-1 ,'whisky',
                   np.where(df['flg_champagne']!=-1, 'champagne',         
                   np.where(df['flg_agua']!=-1, 'agua',
                   np.where(df['flg_jugo']!=-1, 'jugo',
                   np.where(df['flg_fiambre']!=-1, 'fiambre',
                   np.where(df['flg_pan']!=-1, 'panificados',
                   np.where(df['flg_choco']!=-1,'golosinas y chocolates',
                   np.where(df['flg_salchicha']!=-1, 'salchicha',            
                   np.where(df['flg_aderezo']!=-1, 'aderezos',
                   np.where(df['flg_vinagre']!=-1, 'aceites y vinagre',
                   np.where(df['flg_aceite']!=-1, 'aceites y vinagre',
                   np.where(df['flg_goma']!=-1, 'golosinas y chocolates',
                   np.where(df['flg_turron']!=-1, 'golosinas y chocolates',
                   np.where(df['flg_caramelos']!=-1, 'golosinas y chocolates',
                   np.where(df['flg_salsa']!=-1, 'aderezos',
                   np.where(df['flg_alfajor']!=-1, 'golosinas y chocolates',
                   np.where(df['flg_mayonesa']!=-1, 'aderezos',
                np.where(df['flg_ketchup']!=-1, 'aderezos',
                np.where(df['flg_dulce']!=-1, 'golosinas y chocolates',
                np.where(df['flg_gatorade']!=-1, 'jugo',
                np.where(df['flg_espumante']!=-1,'champagne',
                np.where(df['flg_oblea']!=-1,'golosinas y chocolates',
                np.where(df['flg_chupetin']!=-1,'golosinas y chocolates',
                np.where(df['flg_embutido']!=-1,'fiambre',
                np.where(df['flg_aceto']!=-1,'aceites y vinagre',
                np.where(df['flg_mostaza']!=-1,'aderezos',
                np.where(df['flg_postre']!=-1,'golosinas y chocolates',
                np.where(df['flg_mortadela']!=-1, 'quesos',
                np.where(df['flg_huevo']!=-1,'golosinas y chocolates',
                 np.where(df['flg_pastilla']!=-1,'golosinas y chocolates',
                np.where(df['flg_limonada']!=-1,'jugo',
                np.where(df['flg_chicle']!=-1,'golosinas y chocolates',
                np.where(df['flg_jamon']!=-1,'fiambre',
                np.where(df['flg_lacteo']!=-1, 'leches',
                np.where(df['flg_bebida']!=-1,  'jugo',
                np.where(df['flg_bombones']!=-1,  'golosinas y chocolates',
                np.where(df['flg_chimichurri']!=-1,  'aderezos',
                np.where(df['flg_confite']!=-1,  'golosinas y chocolates',
                np.where(df['flg_formula']!=-1, 'leches',
                np.where(df['flg_tableta']!=-1, 'golosinas y chocolates',
                np.where(df['flg_tostada']!=-1, 'panificados',
                np.where(df['flg_rebozador']!=-1,'aderezos',
                np.where(df['flg_salame']!=-1,'salchicha',
                    np.where(df['flg_bocadito']!=-1,'golosinas y chocolates',
                np.where(df['flg_frizze']!=-1,'vino',
                np.where(df['flg_galleta']!=-1,'golosinas y chocolates',
                    np.where(df['flg_bague']!=-1,'panificados',
                         np.where(df['flg_bono']!=-1,'golosinas y chocolates',
                       np.where(df['flg_pan2']!=-1,'panificados',  
                       np.where(df['flg_bondiola']!=-1,'fiambre',
                      np.where(df['flg_garrapi']!=-1,'golosinas y chocolates','otros'
                               ))))))))))))))))))))))))))))))))))))))))))))))))))))))))

df["sub_categoria"]=np.where(df['sub_categoria']!='otros',df['sub_categoria'],
                np.where(df["name"].str.find('soda', start)!=-1, "agua",
                   np.where(df["name"].str.find('gaseo', start)!=-1,"agua",
                np.where(df["name"].str.find('tarte', start)!=-1, 'golosinas y chocolates',
                    np.where(df["name"].str.find('grisi', start)!=-1, 'golosinas y chocolates',
                      np.where(df["name"].str.find('leber', start)!=-1, "salchicha",
                np.where(df["name"].str.find('brah', start)!=-1, 'cervezas',
                   np.where(df["name"].str.find('budwi', start)!=-1,'cervezas',
                    np.where(df["name"].str.find('andes o', start)!=-1,'vino','otros')))))))))

                           
df=df.drop(['flg_leche','flg_queso','flg_yogur'], axis=1)
df=df.drop(['flg_cerveza','flg_vino','flg_whisky','flg_champagne'], axis=1)
df=df.drop(['flg_agua','flg_jugo'], axis=1)
df=df.drop(['flg_fiambre','flg_pan','flg_choco','flg_salchicha','flg_aderezo','flg_vinagre','flg_aceite',
           'flg_chicle','flg_turron','flg_caramelos','flg_salsa'], axis=1)
                          
                                    
            
df["categoria"]=np.where(df['flg_alcohol']==0, 'alcohol',
                   np.where(df['flg_lacteo']==0, 'lacteos',
                   np.where(df['flg_bebida']==0, 'bebidas',
                   np.where(df['flg_almacen']==0, 'almacen','otros'))))

In [75]:
df.head()

,name,description,maker,trademark,flg_espumante,flg_frizze,flg_alcohol,flg_mortadela,flg_lacteo,flg_formula,...,flg_salame,flg_bocadito,flg_bague,flg_bono,flg_pan2,flg_bondiola,flg_garrapi,flg_almacen,sub_categoria,categoria
0,cerveza otro mundo 500 ml,Cerveza Otro Mundo 500 Ml,NaN,Otro Mundo,-1,-1,0,-1,-1,-1,...,-1,-1,-1,-1,-1.0,-1,-1,-1,cervezas,alcohol
1,vino san telmo varietales x750ml.,Vino San Telmo Varietales X750Ml.,NaN,San Telmo,-1,-1,0,-1,-1,-1,...,-1,-1,-1,-1,-1.0,-1,-1,-1,vino,alcohol
2,mayonesa natura 1000cc,Mayonesa Natura con jugo de limon sachet x 100...,gallega,Natura,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1.0,-1,-1,0,aderezos,almacen
3,brahma chopp 354cc x 6,Brahma Chopp 354cc x 6,NaN,Brahma,-1,-1,0,-1,-1,-1,...,-1,-1,-1,-1,-1.0,-1,-1,-1,cervezas,alcohol
4,mayonesa hellmanns light doypack 237 gr,Mayonesa HellmannS Light Doypack 237 Gr,NaN,Hellmann´S,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1.0,-1,-1,0,aderezos,almacen


In [76]:

df_i2= df[["categoria","name"]].groupby(["categoria"]).count().reset_index()
df_i2 = df_i2.rename(columns = {'name': 'ctd_products'}, inplace = False)
df_i2

,categoria,ctd_products
0,alcohol,3322
1,almacen,2711
2,bebidas,1333
3,lacteos,2337
4,otros,297


In [77]:

df_i2= df[["sub_categoria","name"]].groupby(["sub_categoria"]).count().reset_index()
df_i2 = df_i2.rename(columns = {'name': 'ctd_products'}, inplace = False)
df_i2

,sub_categoria,ctd_products
0,aceites y vinagre,290
1,aderezos,516
2,agua,558
3,cervezas,655
4,champagne,296
5,fiambre,119
6,golosinas y chocolates,1404
7,jugo,791
8,leches,678
9,otros,297


# 2. Evaluando modelo

## Asignando código a las categorías

In [78]:
category_mapping = {  0: "aceites y vinagre",
                      1: "aderezos",
                      2: "agua",
                      3: "cervezas",
                      4: "champagne",
                      5: "fiambre",
                      6: "golosinas y chocolates",
                      7: "jugo",
                      8: "leches",
                      9: "panificados",
                      10: "quesos",
                      11: "salchicha",
                      12: "vino",
                      13: "whisky",
                      14: "yogures",
                      15 : "otros",
                   }

reverse_category_mapping = {"aceites y vinagre":0,
                            "aderezos":1,
                            "agua":2,
                            "cervezas":3,
                            "champagne":4,
                            "fiambre":5,
                            "golosinas y chocolates":6,
                            "jugo":7,
                            "leches":8,
                            "panificados":9,
                            "quesos":10,
                            "salchicha":11,
                            "vino":12,
                            "whisky":13,
                            "yogures":14,
                            "otros":15
                           
                           }

In [79]:
def logistic_regression_multiclass(x,y):
  from sklearn.linear_model import LogisticRegression

  #splitting the dataset into training and test parts
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  #bag of words implmentation
  cv = CountVectorizer()
  x_train = cv.fit_transform(x_train).toarray()

  #TF-IDF implementation
  vector = TfidfTransformer()
  x_train = vector.fit_transform(x_train).toarray()
  x_test = cv.transform(x_test)
  
  #Save vector 
  pickle.dump(cv, open("vectorizer.pickle", "wb")) 
    
  reg = 0.1

  #fitting the training dataset to the multiclass classification Logistic Regression model
  lr_model = LogisticRegression(C=1/reg, solver='lbfgs', multi_class='auto', max_iter=1000).fit(x_train,y_train)
  lr_predict = lr_model.predict(x_test)
  lr_pred_prob = lr_model.predict_proba(x_test)

  #evaluation metrics for the dataset
  print("Validation Accuracy: ",accuracy_score(y_test, lr_predict))

  print("\n")
  print("*********** CONFUSION MATRIX **************")
  print(confusion_matrix(y_test,lr_predict))

  print("\n")
  print("*********** CLASSIFICATION REPORT **************")
  print(classification_report(y_test,lr_predict))

  return y_test, lr_predict, lr_pred_prob, lr_model

In [80]:
# the cleaned product description corresponds to the x value 
x = df['description']
# the 13 labels/ product categories mentioned above correspond to the y value
y = df['sub_categoria']

# 3. Results

In [81]:
y_test, lr_predict, lr_pred_prob, lr_model = logistic_regression_multiclass(x,y)

Validation Accuracy:  0.9695


*********** CONFUSION MATRIX **************
[[ 54   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0]
 [  1  94   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 119   0   0   0   1   0   0   0   0   0   0   0   0   0]
 [  0   0   0 129   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  56   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  19   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   4   0 268   0   0   0   0   0   0   1   0   0]
 [  0   0   1   0   0   0   0 150   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 115   0   0   0   0  10   0   0]
 [  0   3   2   0   0   1  12   4   0  51   1   5   0  10   0   0]
 [  0   0   0   0   0   0   1   0   0   0  44   0   0   0   0   0]
 [  0   1   0   0   0   0   0   0   0   0   0 213   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0  31   0   0   0]
 [  1   0   0   0   0   0   0   0   0   1   0   0   0 

## Evaluando en base Test

In [82]:
test = pd.read_csv("test_challenge.csv")

In [83]:
test

,name,description,maker,trademark
0,Queso Sancor Gouda 280 Gr,Queso Sancor Gouda 280 Gr,NaN,Sancor
1,Jugo De Limón Delemon 1 L,Jugo De Limón Delemon 1 L,NaN,Delemon
2,Turrón Georgalos De Yema,Turrón Georgalos De Yema,NaN,Georgalos
3,Ketchup Danica X220Gr.Dp,Ketchup Danica X220Gr.Dp,NaN,La Gallega
4,Queso Reggianito La Paulina Trozado Paquete 1 Kg,Queso Reggianito La Paulina Trozado Paquete 1 Kg,NaN,La Paulina
...,...,...,...,...
908,Prepizza Con Salvado 1 U,Prepizza Con Salvado 1 U,NaN,La Panaderia
909,Cerveza Imperial Scout 500 Cc,Cerveza Imperial Scout 500 Cc,Imperial,Imperial
910,Vino Tinto Humberto Canale Estate Pinot Noir 7...,Vino Tinto Humberto Canale Estate Pinot Noir 7...,NaN,Humberto Canale
911,Leche Entera Vital 3 Nutriplus 200 mL,Leche Entera Vital 3 Nutriplus 200 mL,Danone Argentina,Vital


In [84]:
vector =   pickle.load(open("vectorizer.pickle", 'rb'))   

In [85]:
x = test['description']
x_test = vector.transform(x)

In [86]:
lr_predict = lr_model.predict(x_test)

In [87]:
lr_predict

array(['quesos', 'jugo', 'golosinas y chocolates', 'aderezos', 'quesos',
       'golosinas y chocolates', 'aderezos', 'vino', 'leches', 'vino',
       'vino', 'yogures', 'golosinas y chocolates', 'salchicha', 'agua',
       'panificados', 'quesos', 'yogures', 'aceites y vinagre', 'vino',
       'quesos', 'vino', 'agua', 'golosinas y chocolates', 'champagne',
       'vino', 'vino', 'golosinas y chocolates', 'yogures', 'jugo',
       'vino', 'aderezos', 'agua', 'golosinas y chocolates', 'quesos',
       'quesos', 'vino', 'vino', 'salchicha', 'otros', 'agua', 'yogures',
       'quesos', 'cervezas', 'yogures', 'agua', 'jugo', 'agua',
       'aderezos', 'panificados', 'champagne', 'leches', 'panificados',
       'vino', 'cervezas', 'leches', 'cervezas', 'otros', 'cervezas',
       'jugo', 'jugo', 'jugo', 'cervezas', 'golosinas y chocolates',
       'fiambre', 'leches', 'leches', 'jugo', 'golosinas y chocolates',
       'golosinas y chocolates', 'quesos', 'yogures', 'jugo',
       'aceites y

In [88]:
df_pred =pd.DataFrame(lr_predict)

In [89]:
base_final = pd.concat([test, df_pred], axis=1)

# 3. Results

In [90]:
base_final.to_csv("predictions challenge.csv")

In [91]:
base_final

,name,description,maker,trademark,0
0,Queso Sancor Gouda 280 Gr,Queso Sancor Gouda 280 Gr,NaN,Sancor,quesos
1,Jugo De Limón Delemon 1 L,Jugo De Limón Delemon 1 L,NaN,Delemon,jugo
2,Turrón Georgalos De Yema,Turrón Georgalos De Yema,NaN,Georgalos,golosinas y chocolates
3,Ketchup Danica X220Gr.Dp,Ketchup Danica X220Gr.Dp,NaN,La Gallega,aderezos
4,Queso Reggianito La Paulina Trozado Paquete 1 Kg,Queso Reggianito La Paulina Trozado Paquete 1 Kg,NaN,La Paulina,quesos
...,...,...,...,...,...
908,Prepizza Con Salvado 1 U,Prepizza Con Salvado 1 U,NaN,La Panaderia,otros
909,Cerveza Imperial Scout 500 Cc,Cerveza Imperial Scout 500 Cc,Imperial,Imperial,cervezas
910,Vino Tinto Humberto Canale Estate Pinot Noir 7...,Vino Tinto Humberto Canale Estate Pinot Noir 7...,NaN,Humberto Canale,vino
911,Leche Entera Vital 3 Nutriplus 200 mL,Leche Entera Vital 3 Nutriplus 200 mL,Danone Argentina,Vital,leches
